<a href="https://colab.research.google.com/github/GitMishka/1/blob/main/parseFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

def process_files(csv_file_path, excel_file_path):
    data_csv = pd.read_csv(csv_file_path)
    data_csv['date'] = pd.to_datetime(data_csv['date'])
    filtered_data_csv = data_csv[data_csv['posLabel'].isin(['Director', 'Staff'])]
    unique_employee_counts_csv = filtered_data_csv.groupby(['ComCode', 'posLabel'])['EmployeeId'].nunique().reset_index(name='commEmpCount')

    data_excel = pd.read_excel(excel_file_path)
    data_excel['date'] = pd.to_datetime(data_excel['date'])
    filtered_data_excel = data_excel[data_excel['posLabel'].isin(['Director', 'Staff'])]
    unique_employee_counts_excel = filtered_data_excel.groupby(['comCode', 'posLabel'])['employee_id'].nunique().reset_index(name='happEmpCount')
    avg_score_by_poslabel_excel = filtered_data_excel.groupby(['comCode', 'posLabel'])['score'].mean().reset_index(name='AvgScore')

    combined_results = pd.merge(unique_employee_counts_csv, unique_employee_counts_excel, left_on=['ComCode', 'posLabel'], right_on=['comCode', 'posLabel'], how='outer')
    combined_results_final = pd.merge(combined_results, avg_score_by_poslabel_excel, on=['comCode', 'posLabel'], how='outer').drop(columns=['comCode']).rename(columns={'ComCode': 'comCode'})

    combined_results_final['Participation'] = combined_results_final['happEmpCount'] / combined_results_final['commEmpCount']
    combined_results_final['Participation'] = combined_results_final['Participation'].replace([float('inf'), -float('inf')], np.nan).fillna(0)

    output_file_path = 'final_combined_results_revised.csv'
    combined_results_final.to_csv(output_file_path, index=False)
    print(f'File saved as: {output_file_path}')

csv_file_path = '/path/to/your/3month_list.csv'
excel_file_path = '/path/to/your/happiness_posLabel.xlsx'
process_files(csv_file_path, excel_file_path)
